<a href="https://colab.research.google.com/github/quantumhome/DataAnalysisCaseStudy/blob/master/Silver_Assignment_4_Tool_Chain_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -U langchain langchain-community langchain-huggingface langchain-redis langchain-classic
!service redis-server stop > /dev/null
!apt-get purge redis-server -y > /dev/null
!curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
!echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis-stack.list
!sudo apt-get update > /dev/null
!sudo apt-get install redis-stack-server -y > /dev/null
!redis-stack-server --daemonize yes
!pip install redis > /dev/null

redis-server: unrecognized service
gpg: cannot open '/dev/tty': No such device or address
curl: (23) Failed writing body
deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Starting redis-stack-server, database path /var/lib/redis-stack


In [14]:
import os
from langchain_huggingface import HuggingFaceEmbeddings, ChatHuggingFace, HuggingFaceEndpoint
from langchain_redis import RedisVectorStore
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate

# Fix: Import from langchain_classic for legacy chain support
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain
from google.colab import userdata
# 1. API Configuration
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
REDIS_URL = "redis://localhost:6379"


# 2. Setup Models
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Use ChatHuggingFace wrapper to handle the ChatPromptTemplate correctly
llm_engine = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta", # Changed to a currently supported model
    task="text-generation",
    max_new_tokens=512,
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"]
)
llm = ChatHuggingFace(llm=llm_engine)

# 3. Upload Information to Redis
docs = [
    Document(page_content="The user has successfully bypassed Docker by installing Redis natively.", metadata={"id": 1}),
    Document(page_content="Redis is currently acting as a NoSQL vector database at localhost:6379.", metadata={"id": 2}),
    Document(page_content="LangChain 2025 uses create_retrieval_chain for modern RAG pipelines.", metadata={"id": 3})
]

vector_store = RedisVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    redis_url=REDIS_URL,
    index_name="my_native_index"
)

# 4. Define the Retrieval Logic
# Increased k to get more documents and debug retrieval issues
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

system_prompt = (
    "You are a helpful assistant. Use the following context to answer the question. "
    "If the answer isn't in the context, say you don't know.\n\n"
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

# 5. Assemble and Run the Chain
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# Run the query
response = rag_chain.invoke({"input": "in which port redis is running?"})
print(f"--- LLM Response ---\n{response['answer']}")

--- LLM Response ---


Answer: The specified context states that Redis is currently functioning as a NoSQL vector database on the local network with the IP address 'localhost' and the port number 6379. So, the answer to the question is 6379.
